- Change the np.load to include the file
- Add the following .pkl files into the same folder as this .ipynb (model, model_features, scaler, poly)
- Edit the transform_features() method if necessary

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import entropy
import joblib
import networkx as nx
import sys
import os

sys.path.append(os.path.abspath(".."))

test=np.load("../datasets/labeled/third_batch_multi_labels.npz")

X_test=test["X"]

XX_test = pd.DataFrame(X_test)
XX_test.rename(columns={0:"user",1:"item",2:"rating"},inplace=True)

num_unique_users = XX_test["user"].nunique()
print(f"Number of unique user IDs in the test set: {num_unique_users}")

Number of unique user IDs in the test set: 1100


In [2]:
from utils.feature_transformation import aggregate_features

test_features = aggregate_features(XX_test)
test_features.sort_values(by="user", inplace=True)

# Select only important features
model_features = joblib.load("model_features.pkl")
print(f"test_features before selecting features {test_features.shape}")

# If feature does not exist, populate with 0s
for feat in model_features:
    if feat not in test_features.columns:
        test_features[feat] = 0
test_features = test_features[model_features]
print(f"test_features after selecting features {test_features.shape}")

# debugging
# print(test_features.columns)

scaler = joblib.load("scaler.pkl")
poly = joblib.load("poly.pkl")
test_features_scaled = scaler.transform(test_features)
test_features_poly = poly.transform(test_features_scaled)

# Load the trained model and predict probabilities (shape: #test_users x 6)
xgb_model = joblib.load("model.pkl")
probabilities = xgb_model.predict_proba(test_features_poly)
y_pred_proba_rf = xgb_model.predict_proba(test_features_poly)
print(y_pred_proba_rf)

np.savez("predictions.npz", probabilities=probabilities)
print(f"prediction shape {probabilities.shape}")

test_results=np.load("predictions.npz")
test_results_df = pd.DataFrame(test_results["probabilities"])
test_results_df.head()

test_features before selecting features (1100, 48)
test_features after selecting features (1100, 36)
[[9.8991990e-01 4.4496267e-04 1.9923975e-03 5.6821053e-05 2.6295537e-03
  4.9563562e-03]
 [9.8545176e-01 2.6151519e-03 1.3798808e-03 1.0455541e-04 9.3912734e-03
  1.0573847e-03]
 [7.9256761e-01 1.2256273e-02 1.1199401e-03 8.5607090e-04 1.8873218e-01
  4.4678762e-03]
 ...
 [9.9582541e-01 1.0388228e-03 1.4771029e-04 1.4163861e-04 9.9161186e-04
  1.8548837e-03]
 [9.5830160e-01 5.9007215e-03 1.4022448e-03 1.7019996e-03 1.8766623e-03
  3.0816732e-02]
 [9.9522567e-01 9.1496028e-04 1.8403932e-04 2.4413267e-04 7.5780059e-04
  2.6734071e-03]]
prediction shape (1100, 6)


,0,1,2,3,4,5
0,0.989920,0.000445,0.001992,0.000057,0.002630,0.004956
1,0.985452,0.002615,0.001380,0.000105,0.009391,0.001057
2,0.792568,0.012256,0.001120,0.000856,0.188732,0.004468
3,0.993482,0.004258,0.000195,0.000174,0.001293,0.000597
4,0.996167,0.001729,0.000103,0.000030,0.000177,0.001793


In [3]:
data = np.load('predictions.npz')
predictions = data['probabilities']

class_counts = {i: 0 for i in range(6)}

for row in predictions:
    predicted_class = np.argmax(row)
    class_counts[predicted_class] += 1

print("Class instance counts:")
for class_label, count in class_counts.items():
    print(f"Class {class_label}: {count}") 

Class instance counts:
Class 0: 1003
Class 1: 6
Class 2: 9
Class 3: 28
Class 4: 20
Class 5: 34


In [4]:
from sklearn.metrics import roc_auc_score

y_true = test["y"]

# Convert true labels to a DataFrame
df_y_true = pd.DataFrame(y_true, columns=["user", "true_label"])

# Load the predicted probabilities
predictions_data = np.load("predictions.npz")
probabilities = predictions_data["probabilities"]

predicted_labels = np.argmax(probabilities, axis=1)

df_predictions = pd.DataFrame({
    "user": df_y_true["user"],
    "true_label": df_y_true["true_label"],
    "predicted_label": predicted_labels
})

# Identify misclassified users
df_predictions["correct"] = df_predictions["true_label"] == df_predictions["predicted_label"]
df_misclassified = df_predictions[df_predictions["correct"] == False]
# df_misclassified.head(2)
# df_misclassified.to_csv("misclassified_users.csv", index=False)

auc_per_class = {}
for i in range(probabilities.shape[1]):
    binary_true = (df_predictions["true_label"] == i).astype(int)
    try:
        auc = roc_auc_score(binary_true, probabilities[:, i])
        auc_per_class[i] = auc
        print(f"  Class {i}: AUC = {auc:.3f}")
    except ValueError:
        auc_per_class[i] = None

k = 5
AUC_0 = auc_per_class[0]
anomaly_aucs = [auc_per_class[i] for i in range(1, k+1) if i in auc_per_class]

final_metric = (0.5 * AUC_0) + (0.5 / k) * sum(anomaly_aucs)
print(f"\n🏆 Final Evaluation Metric: {final_metric:.3f}")

# Convert AUC scores to DataFrame
df_auc = pd.DataFrame(list(auc_per_class.items()), columns=["class", "AUC"])


  Class 0: AUC = 0.881
  Class 1: AUC = 0.813
  Class 2: AUC = 0.834
  Class 3: AUC = 0.987
  Class 4: AUC = 0.800
  Class 5: AUC = 0.720

🏆 Final Evaluation Metric: 0.856
